In [ ]:
!pip install afinn
!python -m textblob.download_corpora
!pip install -U textblob
!pip install vaderSentiment
!pip install pyemd
!pip install unidecode

     |████████████████████████████████| 52 kB 826 kB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53447 sha256=a4640a3ba8c1032ac7908edb4cdad37470797245491a07f82f07b6e4f7a3ac2c
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data

In [ ]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from unidecode import unidecode
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  app.launch_new_instance()


# Carregando os embeddings

Aqui vamos utilizar os embeddings para realizar as seguintes atividades:

- análise de simlaridade
- classificação de documentos

<b> Carregue os embeddings treinados, como vimos na Aula 2. É o mesmo arquivo que iremos utilizar</b>

Link: https://drive.google.com/open?id=1zI8pGfbUHuU_0wY_FV4tD6w6ZCUJTQbh

In [ ]:
#opção 1 -> montar o drive no colab e acessar o arquivo de embedding do drive
from google.colab import drive
drive.mount('/content/drive')

#opção 2 -> fazer download e fazer upload por aqui
#from google.colab import files
#uploaded = files.upload()

Mounted at /content/drive


In [ ]:
%%time
# carregar
##na variável path coloque o caminho do embedding baixado:
path = "/content/drive/MyDrive/ptwiki_20180420_100d.txt.bz2"
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(path, binary=False)

CPU times: user 2min 44s, sys: 1.1 s, total: 2min 45s
Wall time: 2min 48s


# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [ ]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado" 
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Distância de Jaccard

<b> Atividade </b>

1) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: lembrando que você precisa aplicar um pre-processamento nessas frases antes de aplicar o método.
Faça:

- Lower
- Remoção StopWords
- Remoção Pontuação
- Tokenização

In [ ]:
def pre_processamento_texto(corpus):
  import nltk
  nltk.download('stopwords')
  print('Document')
  print("#Tokenização")
  corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
  #lowcase
  corpus_alt = [ t.lower() for t in corpus_alt  ]
  print('remove stopwords')
  portuguse_stops = stopwords.words('portuguese')
  corpus_alt = [t for  t in corpus_alt if t not in portuguse_stops]
  print('remove numeros')
  corpus_alt = [re.sub(r'\d','',t) for t in  corpus_alt]
  print('remove pontuação')
  corpus_alt = [t for  t in corpus_alt if t not in string.punctuation]
  #print('remove acentos')
  #corpus_alt = [unidecode(t) for t in  corpus_alt]
  return corpus_alt

In [ ]:
frase_1_pre_processamento=pre_processamento_texto(frase1)
frase_2_pre_processamento=pre_processamento_texto(frase2)
frase_3_pre_processamento=pre_processamento_texto(frase3)
frase_4_pre_processamento=pre_processamento_texto(frase4)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Document
#Tokenização
remove stopwords
remove numeros
remove pontuação
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Document
#Tokenização
remove stopwords
remove numeros
remove pontuação
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Document
#Tokenização
remove stopwords
remove numeros
remove pontuação
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Document
#Tokenização
remove stopwords
remove numeros
remove pontuação


In [ ]:
len(set(frase_1_pre_processamento).intersection(set(frase_2_pre_processamento)))

2

In [ ]:
len(set(frase_1_pre_processamento).union(set(frase_2_pre_processamento)))

19

In [ ]:
def jaccard(f1,f2):
  return len(set(f1).intersection(set(f2)))/len(set(f1).union(set(f2)))

In [ ]:
jaccard(frase_1_pre_processamento,frase_2_pre_processamento)

0.10526315789473684

In [ ]:
jaccard(frase_1_pre_processamento,frase_3_pre_processamento)

0.0

In [ ]:
jaccard(frase_2_pre_processamento,frase_3_pre_processamento)

0.0

In [ ]:
jaccard(frase_1_pre_processamento,frase_4_pre_processamento)

0.0

<b> Atividade </b>

2) Qual par de frase teve maior simlaridade? E qual teve menor? Este resultado faz sentido? Explique.

As frases 1 e 2. As demais tiveram similaridade zero. Não, as frases são utilizadas em contexto parecido, com semântica parecida. O jaccard tem limitações

## Distância de Cosseno

Aqui iremos calcular a distância do cosseno utilizando duas formas, que aprendemos na aula passada, para representar o texto.

- Bag of Words (BOW) 
- Embedding

Observação:

Existem duas formas de trabalhar com o cosseno:

<b> Distância </b>: quanto menor mais perto estão as frases.
<b> Similaridade </b>: quanto maior mais perto estão as frases.

### BOW - Distância do cosseno

<b> Atividade </b>

3) Calcule a distância do cosseno utilizando a representação CountVectorizer e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: no CountVectorizer utilizem as frases já pre-processadas da atividade anterior. Mas para aplicá-las no fit_transform, cada frase deve ser um string (sem estar tokenizada) dentro de uma lista.

```python
#exemplo
distance.cosine(frase1, frase2)
```

In [ ]:
vect_bag = CountVectorizer()

In [ ]:
docs_bag = vect_bag.fit_transform([' '.join(frase_1_pre_processamento),\
                                  ' '.join(frase_2_pre_processamento),\
                                  ' '.join(frase_3_pre_processamento),\
                                  ' '.join(frase_4_pre_processamento)])

In [ ]:
docs_bag[0].todense()

matrix([[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2, 0,
         1, 0, 0, 0, 0, 1, 0, 0]])

In [ ]:
frase1_bow = docs_bag[0].todense()
frase2_bow = docs_bag[1].todense()
frase3_bow = docs_bag[2].todense()
frase4_bow = docs_bag[3].todense()

In [ ]:
print(distance.cosine(frase1_bow, frase2_bow))
print(distance.cosine(frase1_bow, frase3_bow))
print(distance.cosine(frase2_bow, frase3_bow))
print(distance.cosine(frase1_bow, frase4_bow))

0.8309691490542968
1.0
1.0
1.0


### Embedding - Distância do cosseno

Para calcular o embedding de cada uma das frases, utilize o modelo carregado inicialmente. 

Cada palavra tem um vetor, para formar o embedding da frase tire a média de todos os vetores.

Utilize as frases já pre-processadas

<b> Atividade </b> 

4) Calcule a distância do cosseno utilizando a representação Embedding e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4


In [ ]:
word_vectors['fogão']

array([ 0.3057, -0.076 ,  0.6269, -0.2146, -0.4847, -0.1121, -0.132 ,
        0.4016, -0.2735,  0.8309,  0.4079, -0.1401,  0.0916, -0.1943,
       -0.1852, -0.1271,  0.124 , -0.2686, -0.0234, -0.0617, -0.5043,
       -0.0357, -0.3085, -0.1748,  0.0502,  0.2168, -0.5501, -0.4325,
       -0.3013, -0.3772,  0.8357,  0.0859,  0.6692,  1.1012, -0.1948,
       -0.4732, -0.1394, -0.4083,  0.6847,  0.0633, -0.1836, -0.1726,
       -0.4396, -0.0672,  0.5016,  0.0675, -0.2919, -0.0864, -0.6857,
       -0.4937,  0.8398, -0.2043, -0.4746,  0.0547,  0.041 ,  0.0535,
       -0.0022, -0.8022,  0.5122,  0.3449,  0.7115, -0.8373, -0.2469,
       -0.1739,  0.0985,  0.4376, -0.0918, -0.0374,  0.0578,  0.2694,
       -0.0275, -0.2675, -0.5567, -0.3373, -0.527 , -0.0812,  0.1699,
       -0.3703, -0.6308, -0.6002,  0.4416, -0.8358, -0.174 ,  0.0079,
        0.1091,  0.2028,  0.2061, -0.0552, -0.0064,  0.6256,  0.8936,
       -0.313 , -0.365 , -0.4255, -0.5043,  0.4329,  0.6975, -0.2431,
        0.0726, -0.3

In [ ]:
word_vectors['ótimo']

array([ 0.07  ,  0.328 , -0.4575,  0.0816, -0.5353, -0.2608,  0.18  ,
        0.3187, -0.4454,  0.3796,  0.4828, -0.225 , -0.3291,  0.1281,
       -0.059 ,  0.5279,  0.2425, -0.8551,  0.2208,  0.2369, -0.1975,
        0.3723, -0.126 , -0.0727, -0.5586,  0.0239, -0.1535, -0.7805,
        0.1168,  0.1669, -0.0682,  0.184 ,  0.5678,  0.3104, -0.1147,
       -0.5562,  0.0407, -0.064 ,  0.0974,  0.1565,  0.5655, -0.0801,
       -0.3852,  0.322 ,  0.0552, -0.0832, -0.0416,  0.2012, -0.9991,
        0.0058,  0.7823,  0.3251,  0.0067,  0.1056,  0.6211, -0.4179,
       -0.0079, -0.5724,  0.1751,  0.3493,  0.3302, -0.5236, -0.1005,
        0.4321,  0.4189, -0.0374, -0.2467,  0.1282,  0.4268,  0.0969,
       -0.3075,  0.5029, -0.2893, -0.2478, -0.0493,  0.4143, -0.2963,
        0.279 ,  0.005 ,  0.2045, -0.158 ,  0.2136, -0.5662, -0.1397,
       -0.6357, -0.3585, -0.0588, -0.4662,  0.2503, -0.678 ,  0.8295,
       -0.5559, -0.2378, -0.1326, -0.0595, -0.0442, -0.0446, -0.1188,
       -0.7148, -0.1

In [ ]:
np.mean(np.array([word_vectors[palavra] for palavra in frase_4_pre_processamento]),axis=0)

array([ 0.18785   ,  0.126     ,  0.0847    , -0.06649999, -0.51      ,
       -0.18645   ,  0.024     ,  0.36014998, -0.35944998,  0.60525   ,
        0.44535   , -0.18255   , -0.11875001, -0.0331    , -0.1221    ,
        0.2004    ,  0.18325001, -0.56184995,  0.0987    ,  0.0876    ,
       -0.3509    ,  0.1683    , -0.21724999, -0.12375   , -0.2542    ,
        0.12035   , -0.35180002, -0.6065    , -0.09224999, -0.10515001,
        0.38375   ,  0.13495   ,  0.6185    ,  0.7058    , -0.15475   ,
       -0.5147    , -0.04935   , -0.23615001,  0.39105   ,  0.1099    ,
        0.19095   , -0.12635   , -0.4124    ,  0.1274    ,  0.2784    ,
       -0.00785   , -0.16675   ,  0.0574    , -0.8424    , -0.24395   ,
        0.81105   ,  0.0604    , -0.23394999,  0.08015   ,  0.33105   ,
       -0.1822    , -0.00505   , -0.68729997,  0.34364998,  0.34710002,
        0.52085   , -0.68044996, -0.1737    ,  0.1291    ,  0.2587    ,
        0.20009999, -0.16925   ,  0.0454    ,  0.2423    ,  0.18

In [ ]:
def calcula_embedding(frase):
  return np.mean(np.array([word_vectors[palavra] for palavra in frase]),axis=0)

In [ ]:
frase1_emb = calcula_embedding(frase_1_pre_processamento)
frase2_emb = calcula_embedding(frase_2_pre_processamento)
frase3_emb = calcula_embedding(frase_3_pre_processamento)
frase4_emb = calcula_embedding(frase_4_pre_processamento)

In [ ]:
print(distance.cosine(frase1_emb, frase2_emb))
print(distance.cosine(frase1_emb, frase3_emb))
print(distance.cosine(frase2_emb, frase3_emb))
print(distance.cosine(frase1_emb, frase4_emb))

0.16096973419189453
0.2234203815460205
0.2526938319206238
0.29043054580688477


<b>Atividade </b>

5) Qual diferença entre as representações do exercício 3 e 4? Qual faz mais sentido? Explique.

A distância de similaridade de cosseno foi cálculada com bag of words e outra com embedding. A cálculada com embedding pois representa melhor as frases com contexto semelhante.

## WMD

O WMD já está incorporado ao Word2Vec

<b> Atividade </b>

6) Calcule a distância WMD e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: use a variável já tokenizada.

In [ ]:
print(word_vectors.wmdistance(frase_1_pre_processamento,frase_2_pre_processamento))
print(word_vectors.wmdistance(frase_1_pre_processamento,frase_3_pre_processamento))
print(word_vectors.wmdistance(frase_2_pre_processamento,frase_3_pre_processamento))
print(word_vectors.wmdistance(frase_1_pre_processamento,frase_4_pre_processamento))

3.0725697404938526
3.759219170455064
3.9048993635316385
3.851623338336251


<b> Atividade </b>

7) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.

A frase 2 e 3. A frase 1 com a 2. Sim. As frases mais parecidas tem a menor distância WMD.

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Dataset

<b> Atividade </b>

8) Carregue o dataset com o pandas e depois dê o head no dataframe.


Link download: https://drive.google.com/open?id=15azJWdEEPGsXQGiDmEOseTBJcquWvBQc

<b> Este dataset é sobre revisões de filmes do IMDB. </b>

In [ ]:
%%time
# carregar
##na variável path coloque o caminho do embedding baixado:
path_imdb = "/content/drive/MyDrive/Trabalhos - Pós/imdb-reviews-pt-br.csv"

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 19.6 µs


In [ ]:
df= pd.read_csv(path_imdb)

In [ ]:
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


## Representação dos dados

O sentimento positivo e negativo iremos binarizar cada um deles. Seja 1 positivo e 0 negativo.

Iremos representar o texto de duas formas:

- Bag of Words (BOW)
- Embedding

Depois iremos comparar o resultado de cada um deles.

### Representação Target

<b> Atividade </b>

9) Faça a representação dos sentimentos. 1 positivo; 0 negativo

In [ ]:
target = df['sentiment'].replace(['neg','pos'],[0,1])

### Bag of Words (BOW)

<b> Atividade </b>

10) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords``` no dataframe para armazenar este dado processado):

- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto sem tokenização, ou seja uma string

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
def pre_processamento_texto(corpus):
  import nltk
  nltk.download('stopwords')
  print('Document')
  print("#Tokenização")
  corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
  #lowcase
  corpus_alt = [ t.lower() for t in corpus_alt  ]
  print('remove stopwords')
  portuguse_stops = stopwords.words('portuguese')
  corpus_alt = [t for  t in corpus_alt if t not in portuguse_stops]
  print('remove numeros')
  corpus_alt = [re.sub(r'\d','',t) for t in  corpus_alt]
  print('remove pontuação')
  corpus_alt = [t for  t in corpus_alt if t not in string.punctuation]
  #print('remove acentos')
  #corpus_alt = [unidecode(t) for t in  corpus_alt]
  corpus_alt = ' '.join(corpus_alt).lower()
  return corpus_alt

In [ ]:
df['text_pt_LIMPO_BOW'] = df['text_pt'].progress_apply(pre_processamento_texto) 

In [ ]:
df['text_pt_LIMPO_BOW']

0        vez sr costner arrumou filme tempo necessário ...
1        exemplo motivo maioria filmes ação mesmos gené...
2        primeiro tudo odeio raps imbecis poderiam agir...
3        beatles puderam escrever músicas todos gostass...
4        filmes fotos latão palavra apropriada verdade ...
                               ...                        
49454    média votos baixa fato funcionário locadora ac...
49455    enredo algumas reviravoltas infelizes inacredi...
49456    espantado forma filme maioria outros média est...
49457    christmas together realmente veio antes tempo ...
49458    drama romântico classe trabalhadora diretor ma...
Name: text_pt_LIMPO_BOW, Length: 49459, dtype: object

<b> Atividade </b>

11) Aplique a representação do texto processado anteriormente com CountVectorizer.

In [ ]:
vect_bag = CountVectorizer()

In [ ]:
X_bag = vect_bag.fit_transform(df['text_pt_LIMPO_BOW'])

In [ ]:
df.shape

(49459, 6)

In [ ]:
X_bag

<49459x127503 sparse matrix of type '<class 'numpy.int64'>'
	with 5030933 stored elements in Compressed Sparse Row format>

### Embedding

<b> Atividade </b>

12) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords_token``` no dataframe para armazenar este dado processado):

- Aplique lower
- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto com tokenização

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def pre_processamento_texto(corpus):
  #print("#Tokenização")
  corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
  #lowcase
  corpus_alt = [ t.lower() for t in corpus_alt  ]
  #print('remove stopwords')
  portuguse_stops = stopwords.words('portuguese')
  corpus_alt = [t for  t in corpus_alt if t not in portuguse_stops]
  #print('remove numeros')
  corpus_alt = [re.sub(r'\d','',t) for t in  corpus_alt]
  #print('remove pontuação')
  corpus_alt = [t for  t in corpus_alt if t not in string.punctuation]
  return corpus_alt

In [ ]:
df['text_pt_LIMPO_EMB'] = df['text_pt'].progress_apply(lambda x: pre_processamento_texto(x)) 

  0%|          | 0/49459 [00:00<?, ?it/s]

In [ ]:
df['text_pt_LIMPO_EMB']

0        [vez, sr, costner, arrumou, filme, tempo, nece...
1        [exemplo, motivo, maioria, filmes, ação, mesmo...
2        [primeiro, tudo, odeio, raps, imbecis, poderia...
3        [beatles, puderam, escrever, músicas, todos, g...
4        [filmes, fotos, latão, palavra, apropriada, ve...
                               ...                        
49454    [média, votos, baixa, fato, funcionário, locad...
49455    [enredo, algumas, reviravoltas, infelizes, ina...
49456    [espantado, forma, filme, maioria, outros, méd...
49457    [christmas, together, realmente, veio, antes, ...
49458    [drama, romântico, classe, trabalhadora, diret...
Name: text_pt_LIMPO_EMB, Length: 49459, dtype: object


<b> Atividade </b>

13) Aplique a representação do texto com Embeddings. Cada palavra tem um embedding, o embedding da frase é a média de todos embeddings.

In [ ]:
def calcula_embedding(frase):
  return np.mean(np.array([word_vectors[palavra] for palavra in frase if palavra in word_vectors.vocab]),axis=0)

In [ ]:
df['text_pt_LIMPO_EMB']

0        [vez, sr, costner, arrumou, filme, tempo, nece...
1        [exemplo, motivo, maioria, filmes, ação, mesmo...
2        [primeiro, tudo, odeio, raps, imbecis, poderia...
3        [beatles, puderam, escrever, músicas, todos, g...
4        [filmes, fotos, latão, palavra, apropriada, ve...
                               ...                        
49454    [média, votos, baixa, fato, funcionário, locad...
49455    [enredo, algumas, reviravoltas, infelizes, ina...
49456    [espantado, forma, filme, maioria, outros, méd...
49457    [christmas, together, realmente, veio, antes, ...
49458    [drama, romântico, classe, trabalhadora, diret...
Name: text_pt_LIMPO_EMB, Length: 49459, dtype: object

In [ ]:
X_embedding = df['text_pt_LIMPO_EMB'].progress_apply(lambda x: calcula_embedding(x)) 

  0%|          | 0/49459 [00:00<?, ?it/s]

In [ ]:
X_embedding

0        [0.14301138, 0.22025318, -0.047075942, -0.2170...
1        [0.16303213, 0.22678684, -0.05451168, -0.20547...
2        [0.21659917, 0.14430527, -0.0034280657, -0.197...
3        [0.2596114, 0.13605723, -0.07343373, -0.208256...
4        [0.22052252, 0.087914094, -0.06986406, -0.1659...
                               ...                        
49454    [0.16023268, 0.1474827, -0.08474423, -0.103290...
49455    [0.17767419, 0.1423968, -0.081170976, -0.22615...
49456    [0.18305564, 0.18313898, -0.062747374, -0.1887...
49457    [0.18229564, 0.15024172, -0.08812309, -0.22163...
49458    [0.26787597, 0.16923124, -0.06144687, -0.18159...
Name: text_pt_LIMPO_EMB, Length: 49459, dtype: object

## Treinamento

### CountVectorizer


<b> Atividade </b>

14) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

```python
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)
```

In [ ]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)


<b> Atividade </b>

15) Treine com uma regressão logística

In [ ]:
modelo_bow = LogisticRegression()
modelo_bow.fit(X_train_bow,y_train_bow)

LogisticRegression()


<b> Atividade </b>

16) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [ ]:
y_predi = modelo_bow.predict(X_test_bow)

In [ ]:
y_predi

array([1, 1, 1, ..., 0, 1, 1])

In [ ]:
print(classification_report(y_test_bow, y_predi))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88      6112
           1       0.87      0.89      0.88      6253

    accuracy                           0.88     12365
   macro avg       0.88      0.88      0.88     12365
weighted avg       0.88      0.88      0.88     12365



### Embedding


<b> Atividade </b>

17) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

Verifique o shape do X treino e X teste. Caso eles estejam com apenas uma dimensão, você precisa tranformá-los para duas dimensões, caso contrário ocorrerá erro no treinamento.

In [ ]:
X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_embedding.values,target,random_state=123)

In [ ]:
X_train_emb = pd.DataFrame([x for x in X_train_emb])

In [ ]:
X_test_emb = pd.DataFrame([x for x in X_test_emb])


<b> Atividade </b>

18) Treine com uma regressão logística

In [ ]:
modelo_bow = LogisticRegression()
modelo_bow.fit(X_train_emb, y_train_emb)

LogisticRegression()


<b> Atividade </b>

19) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [ ]:
y_predi = modelo_bow.predict(X_test_emb)

In [ ]:
print(classification_report(y_test_emb, y_predi))

              precision    recall  f1-score   support

           0       0.76      0.77      0.77      6112
           1       0.78      0.77      0.77      6253

    accuracy                           0.77     12365
   macro avg       0.77      0.77      0.77     12365
weighted avg       0.77      0.77      0.77     12365



#### Calcule as métricas


<b> Atividade </b>

20) Compare os resultados obtidos com o BagOfWords e com o Embedding. Explique os possíveis motivos desta diferença.

O problema e de análise sentimentos, a variável target é positivo ou negativo. O bag of words demarca a presença ou não das palavras dentro de um texto. Ou ele mapeia melhor as palavras negativas e positivas de um texto que o embedding. Para o problema de análise de sentimentos com a base de dados em questão foi melhor o Bag of word do que o embedding analisando a semântica do contexto. O embedding apesar de uma boa metodologia, foi utilizado tem 100 dimensões enquanto que o Bag of words tem 127000

# Análise de sentimentos

O modelo que criamos anteriormente é para ilustrar como podemos realizar classificação de documentos.
Quando a tarefa é sobre análise de sentimentos, temos duas opções: treinar nosso próprio modelo, como feito anteriormente ou utilizar uma das inúmeras ferramentas prontas.

Vamos testar as seguintes ferramentas:

- Vader
- Textblob
- Affin

Nesta atividade iremos utilizar as duas variáveis abaixo:

In [ ]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

## Vader

<b> Apenas Inglês </b>

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment


<b> Atividade </b>

21) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto)
```

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto_neg)

{'compound': 0.3958, 'neg': 0.126, 'neu': 0.76, 'pos': 0.114}

In [ ]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto_pos)

{'compound': 0.9969, 'neg': 0.084, 'neu': 0.737, 'pos': 0.179}

## TextBlob

<b> Apenas inglês </b>

https://www.presentslide.in/2019/08/sentiment-analysis-textblob-library.html

<b> Atividade </b>
 
22) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
sentence=TextBlob(texto)
sentence.sentiment
```

In [ ]:
from textblob import TextBlob

In [ ]:
sentence=TextBlob(texto_neg )

In [ ]:
sentence.sentiment

Sentiment(polarity=0.06385964912280702, subjectivity=0.5629824561403508)

In [ ]:
sentence=TextBlob(texto_pos)

In [ ]:
sentence.sentiment

Sentiment(polarity=0.190819118692253, subjectivity=0.6026226012793177)

## Afinn

- Valor maior que 0 indica sentimento positivo
- Valor menor que 0 indica sentimento negativo

<b> Atividade </b>

23) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
afinn = Afinn()
afinn.score(texto)
```

In [ ]:
from afinn import Afinn

In [ ]:
afinn = Afinn()

In [ ]:
afinn.score(texto_neg)

8.0

In [ ]:
afinn.score(texto_pos)

55.0

<b> Atividade </b>

24) Para você, qual ferramenta teve melhor comportamento?  Justifique com exemplos.

O sentimento de Vader pois retorna a probabilidade de uma determinada frase de entrada ser positiva, negativa e neutra.

Por exemplo:

“O filme foi incrível!”
Positivo: 99%
Negativo: 1%
Neutro: 0%

# Dica:
## Quando for trabalhar com um dataset em inglês, a biblioteca Spacy facilita!

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy
import pandas as pd

O scpay forne um pacote que já tem série de modelos já treinados em NLP. Inclusive para os embeddings em inglês.

Para mais informações vá em:

https://spacy.io/models/en#en_core_web_md



Com o método abaixo carregamos um dos modelos do spacy:

```python
nlp = spacy.load('en_core_web_md')
```

Para aplicar o modelo, basta passar o texto para o modelo carregado anteriormente:

```python
doc = nlp("This is some text that I am processing with Spacy")
```

Carregue o modelo e imprima doc

In [ ]:
nlp = spacy.load('en_core_web_md')

In [ ]:
doc = nlp("This is some text that I am processing with Spacy")

In [ ]:
doc

Ao aplicar o modelo carregado a variável <b> doc </b> já possui os embeddings de cada uma das palavras e o embedding da frase, que é a média de todos vetores de todas palavras.

```python
#vetor da primeira palavra
doc[0].vector
#vetor agregado pela média - embedding do documento
doc.vector
```
O código abaixo mostra que a média de uma posição em específico dos embeddings de todas as palavras e a posição do embedding do documento possuem o mesmo valor.

In [ ]:
def calcula_media_posicao(x):
    soma = 0
    vector = []
    for i in range(0,len(doc)):
        vector.append(doc[i].vector)    
    
    for v in vector:
        soma += v[x]
    return soma/len(doc)

In [ ]:
round(calcula_media_posicao(10),6)

In [ ]:
round(doc.vector[10], 6)